In [ ]:
!pip install flash-attn --no-build-isolation
!pip install git+https://github.com/TIGER-AI-Lab/Mantis.git

: 

In [ ]:
import torch
print(torch.version.cuda)

In [ ]:
from mantis.models.mllava import chat_mllava
from PIL import Image
import flash_attn_2_cuda
import torch
import json
from transformers import AutoProcessor, AutoModelForCausalLM, GenerationConfig, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model


In [ ]:
!pip install --upgrade transformers
!pip uninstall bitsandbytes
!pip install bitsandbytes
!pip install transformers bitsandbytes torch
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)        # Should match your CUDA version

In [ ]:
from mantis.models.mllava import chat_mllava
from PIL import Image
import json
from transformers import AutoProcessor, AutoModel, BitsAndBytesConfig

# Define the model path
model_path = "TIGER-Lab/Mantis-8B-Idefics2"  # Replace with the correct model path

# Load the processor
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

# Load the model with 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModel.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

In [ ]:
from mantis.models.mllava import chat_mllava
from PIL import Image
import flash_attn_2_cuda
import json
from transformers import AutoProcessor, AutoModelForCausalLM, GenerationConfig, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# Example usage of GenerationConfig (if needed)
generation_config = GenerationConfig(
    max_length=128,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

print("GenerationConfig imported and defined successfully.")

In [ ]:
from mantis.models.mllava import chat_mllava
from PIL import Image
import flash_attn_2_cuda
import json
from transformers import AutoProcessor, AutoModelForCausalLM, GenerationConfig, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# Define LoRA configuration
peft_config = LoraConfig(
    r=8,  # Rank of the LoRA update matrices
    lora_alpha=32,  # Scaling factor for LoRA updates
    target_modules=["q_proj", "v_proj"],  # Target modules to apply LoRA
    lora_dropout=0.1,  # Dropout probability for LoRA
    bias="none",  # Whether to add bias
    task_type="CAUSAL_LM"  # Task type (e.g., causal language modeling)
)

print("LoRA configuration defined successfully.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration
from PIL import Image
import os
import zipfile

# Define the path to the zip file
zip_file_path = "/png.zip"  # Replace with the actual path to your zip file
extracted_folder = "/extracted_images"  # Folder to extract images

# Extract the zip file
os.makedirs(extracted_folder, exist_ok=True)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

print(f"Extracted images to {extracted_folder}")

# %% Load JSON file for processing
input_file = "/test_human.json"  # Replace with your input JSON file path
output_file = "output.json"  # File to save generated answers

with open(input_file, "r") as f:
    data = json.load(f)

# %% Initialize Pix2Struct for chart QA
processor = Pix2StructProcessor.from_pretrained('google/deplot')
model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')

# Process each entry in the JSON file
previous_file_name = ""

# List to store results
results = []

# %% Iterate through each entry in the JSON file
for entry in data:
    file_name = entry["imgname"]
    file_path = "/extracted_images/png/{}".format(file_name)
    question = entry["query"]
    expected_answer = entry["label"]



In [ ]:
import matplotlib.pyplot as plt
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration
from PIL import Image
import os

# Initialize variables
previous_file_name = ""
results = []

# Initialize loss tracking
losses = []
epochs = []

# Counters for accuracy
correct_human = 0
total_human = 0
correct_augmented = 0
total_augmented = 0

# Simulate multiple epochs (for demonstration purposes)
for epoch in range(1, 6):  # Simulate 5 epochs
    epoch_loss = 0  # Track total loss for the epoch
    epoch_correct = 0  # Track correct predictions for the epoch
    epoch_total = 0  # Track total predictions for the epoch

    # Iterate through each entry in the dataset (assuming `data` is a list of entries)
    for entry in data:
        file_name = entry["imgname"]
        file_path = "/extracted_images/png/{}".format(file_name)  # Replace with the actual path to your images
        question = entry["query"]
        expected_answer = entry["label"]

        # Reload pipeline if the file name changes
        if file_name != previous_file_name:
            print(f"Loading new image: {file_name}")
            processor = Pix2StructProcessor.from_pretrained('google/deplot')
            model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')
            previous_file_name = file_name  # Update tracking variable

        # Open the image and verify it's valid
        try:
            image = Image.open(file_path)
            width, height = image.size  # Validate image dimensions (width and height must not be None)
            if width is None or height is None:
                print(f"Invalid image dimensions for {file_path}")
                continue  # Skip this image if dimensions are invalid
        except (IOError, SyntaxError) as e:
            print(f"Error loading image {file_path}: {e}")
            continue  # Skip this image and move to the next one

        # Process the image and query
        inputs = processor(images=image, text="Using the chart provided, answer the question: {}".format(question), return_tensors="pt")
        predictions = model.generate(**inputs, max_new_tokens=512)
        generated_answer = processor.decode(predictions[0], skip_special_tokens=True)

        print("Question:", question)
        print("Generated Answer:", generated_answer)
        print("Expected Answer:", expected_answer)

        # Calculate accuracy
        is_correct = generated_answer.strip().lower() == expected_answer.strip().lower()
        epoch_total += 1
        if is_correct:
            epoch_correct += 1

        # Simulate a loss metric (e.g., string similarity or token overlap)
        loss = 1.0 if not is_correct else 0.0  # Loss is 1 if incorrect, 0 if correct
        epoch_loss += loss

        # Track results for test_human and test_augmented
        if "human" in file_name:  # Assuming filenames indicate the dataset
            total_human += 1
            if is_correct:
                correct_human += 1
        elif "augmented" in file_name:  # Assuming filenames indicate the dataset
            total_augmented += 1
            if is_correct:
                correct_augmented += 1

        # Store the result
        results.append({
            "imgname": file_name,
            "query": question,
            "label": expected_answer,
            "generated_answer": generated_answer,
            "is_correct": is_correct,
            "loss": loss
        })

    # Track epoch loss and accuracy
    losses.append(epoch_loss / epoch_total)  # Average loss for the epoch
    epochs.append(epoch)
    print(f"Epoch {epoch}: Loss = {losses[-1]:.4f}, Accuracy = {epoch_correct / epoch_total * 100:.2f}%")

# Calculate overall accuracy
total_overall = total_human + total_augmented
correct_overall = correct_human + correct_augmented
accuracy_human = correct_human / total_human if total_human > 0 else 0
accuracy_augmented = correct_augmented / total_augmented if total_augmented > 0 else 0
accuracy_overall = correct_overall / total_overall if total_overall > 0 else 0

# Print accuracies
print(f"Accuracy on test_human: {accuracy_human * 100:.2f}%")
print(f"Accuracy on test_augmented: {accuracy_augmented * 100:.2f}%")
print(f"Overall accuracy: {accuracy_overall * 100:.2f}%")

# Save results to a JSON file
output_file = "output_results.json"
with open(output_file, "w") as f:
    json.dump(results, f, indent=4)

print(f"Results saved to {output_file}")

# Plot losses over epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, losses, label="Loss", marker='o')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss Over Time")
plt.legend()
plt.grid()
plt.show()